In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
org_data = pd.read_csv("rule_based_data.csv",index_col=False)
if "Unnamed: 0" in org_data.columns:
    org_data = org_data.drop(columns=["Unnamed: 0"])

In [3]:
org_data

,CROPS,TYPE_OF_CROP,SOIL,SEASON,SOWN,HARVESTED,SOIL_PH,CROP_DURATION,TEMP,WATER_SOURCE,WATER_REQUIRED,RELATIVE_HUMIDITY
0,rice,cereals,Alluvia or loamy and clayey soil,kharif,Jun-Jul,Sep-Oct,6.0,150,26.85,"irrigated,rainfall",1869,79.94
1,rice,cereals,Alluvia or loamy and clayey soil,kharif,Jun-Jul,Sep-Oct,7.6,150,36.32,"irrigated,rainfall",959,60.38
2,rice,cereals,Alluvia or loamy and clayey soil,kharif,Jun-Jul,Sep-Oct,7.8,150,22.20,"irrigated,rainfall",1035,77.97
3,rice,cereals,Alluvia or loamy and clayey soil,kharif,Jun-Jul,Sep-Oct,7.6,150,20.44,"irrigated,rainfall",1704,66.53
4,rice,cereals,Alluvia or loamy and clayey soil,kharif,Jun-Jul,Sep-Oct,6.7,150,32.04,"irrigated,rainfall",1464,78.64
...,...,...,...,...,...,...,...,...,...,...,...,...
565,small onion,bulbvegetables,Sandy loam,Zaid,Mar-Jul,Mar-Jul,6.8,78,16.33,"irrigated,rainfall",702,69.91
566,small onion,bulbvegetables,Sandy loam,Zaid,Mar-Jul,Mar-Jul,6.4,90,24.99,"irrigated,rainfall",676,72.94
567,small onion,bulbvegetables,Sandy loam,Zaid,Mar-Jul,Mar-Jul,6.5,72,17.48,"irrigated,rainfall",710,72.36
568,small onion,bulbvegetables,Sandy loam,Zaid,Mar-Jul,Mar-Jul,6.9,76,23.58,"irrigated,rainfall",711,71.49


In [4]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

synthetic_df=pd.DataFrame()
for i in range(57):
    print(i)
    if i==0:
        crop_data = org_data[(i*10):(i+1)*10]
    else:
        crop_data = org_data[(i*10)+1:(i+1)*10]
    # Extract numerical columns
    numerical_columns = ["SOIL_PH", "CROP_DURATION", "TEMP", "WATER_REQUIRED", "RELATIVE_HUMIDITY"]
    categorical_columns = ["CROPS", "WATER_SOURCE", "SOIL","TYPE_OF_CROP","SEASON","SOWN","HARVESTED"]

    # Create a MinMaxScaler
    scaler = MinMaxScaler()
    # Normalize only numerical columns
    crop_data[numerical_columns] = scaler.fit_transform(crop_data[numerical_columns])

    # Perform one-hot encoding for categorical columns
    encoder = OneHotEncoder(sparse=False, drop='first')
    categorical_encoded = encoder.fit_transform(crop_data[categorical_columns])
    categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names(categorical_columns))

    # Concatenate the encoded categorical columns with the normalized numerical columns
    normalized_data = pd.concat([crop_data[numerical_columns], categorical_encoded_df], axis=1)
    # GAN parameters
    latent_dim = 100
    num_samples = len(normalized_data)
    epochs = 1000
    batch_size = 64
    # Generator model
    generator = keras.Sequential([
        layers.Input(shape=(latent_dim,)),
        layers.Dense(256, activation="relu"),
        layers.Dense(512, activation="relu"),
        layers.Dense(len(normalized_data.columns), activation="sigmoid")  # Output layer with same dimensions as input
    ])
    # Discriminator model
    discriminator = keras.Sequential([
        layers.Input(shape=(len(normalized_data.columns),)),
        layers.Dense(512, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(1, activation="sigmoid")  # Binary classification output
    ])
    # GAN model (combining generator and discriminator)
    discriminator.compile(loss="binary_crossentropy", optimizer="adam")
    discriminator.trainable = False
    gan_input = keras.Input(shape=(latent_dim,))
    gan_output = discriminator(generator(gan_input))
    gan = keras.Model(gan_input, gan_output)
    gan.compile(loss="binary_crossentropy", optimizer="adam")

    # Training loop
    for epoch in range(epochs):
        noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
        generated_data = generator.predict(noise)

        real_data_indices = np.random.choice(len(crop_data), batch_size)
        real_data = crop_data.drop(columns=["CROPS","WATER_SOURCE","SOIL","TYPE_OF_CROP","SEASON","SOWN","HARVESTED"]).iloc[real_data_indices].values.astype(np.float32)

        real_data_label = np.ones((batch_size, 1), dtype=np.float32)
        fake_data_label = np.zeros((batch_size, 1), dtype=np.float32)

        # Train discriminator
        d_loss_real = discriminator.train_on_batch(real_data, real_data_label)
        d_loss_fake = discriminator.train_on_batch(generated_data, fake_data_label)

        # Train generator (via GAN model)
        noise = np.random.normal(0, 1, size=(batch_size, latent_dim)).astype(np.float32)
        g_loss = gan.train_on_batch(noise, real_data_label)

        if epoch % 100 == 0:
            print(f"Epoch {epoch}: D Loss Real: {d_loss_real:.4f}, D Loss Fake: {d_loss_fake:.4f}, G Loss: {g_loss:.4f}")
    
    # Generate synthetic crop data
    num_synthetic_samples = 100
    noise = np.random.normal(0, 1, size=(num_synthetic_samples, latent_dim))
    synthetic_data = generator.predict(noise)
    synthetic_data_denormalized = (synthetic_data * (scaler.data_max_ - scaler.data_min_)) + scaler.data_min_
    
    for col in normalized_data.columns:
        col_min = org_data[col].min()
        col_max = org_data[col].max()
        synthetic_data_denormalized[:, normalized_data.columns.get_loc(col)] = np.clip(
            synthetic_data_denormalized[:, normalized_data.columns.get_loc(col)], col_min, col_max
        )
    
    decimal_places = {"SOIL_PH": 1, "CROP_DURATION": 0, "TEMP": 2, "WATER_REQUIRED": 0, "RELATIVE_HUMIDITY": 2}
    synthetic_data_denormalized_rounded = synthetic_data_denormalized.copy()
    for column, places in decimal_places.items():
        synthetic_data_denormalized_rounded[:, normalized_data.columns.get_loc(column)] = np.round(synthetic_data_denormalized[:, normalized_data.columns.get_loc(column)], places)
    

    epoch_synthetic_df = pd.DataFrame(synthetic_data_denormalized_rounded, columns=normalized_data.columns)
    epoch_synthetic_df.insert(0, 'CROPS', org_data["CROPS"][(10*i)+1])
    epoch_synthetic_df.insert(1, 'TYPE_OF_CROP', org_data["TYPE_OF_CROP"][(10*i)+1])
    epoch_synthetic_df.insert(2, 'SOIL', org_data["SOIL"][(10*i)+1])
    epoch_synthetic_df.insert(3, 'SEASON', org_data["SEASON"][(10*i)+1])
    epoch_synthetic_df.insert(4, 'SOWN', org_data["SOWN"][(10*i)+1])
    epoch_synthetic_df.insert(5, 'HARVESTED', org_data["HARVESTED"][(10*i)+1])
    epoch_synthetic_df.insert(8, 'WATERSOURCE', org_data["WATER_SOURCE"][(10*i)+1])
    synthetic_df = synthetic_df.append(epoch_synthetic_df, ignore_index=True)
    

0


c:\Users\lalit\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


2/2 [==============================] - 0s 3ms/step
Epoch 0: D Loss Real: 0.6329, D Loss Fake: 0.8609, G Loss: 0.5762
1/1 [==============================] - 0s 53ms/step
1
2/2 [==============================] - 0s 3ms/step


c:\Users\lalit\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 0: D Loss Real: 0.6576, D Loss Fake: 0.8492, G Loss: 0.5788
1/1 [==============================] - 0s 47ms/step
2
2/2 [==============================] - 0s 2ms/step


...

Epoch 0: D Loss Real: 0.6786, D Loss Fake: 0.7903, G Loss: 0.6314
1/1 [==============================] - 0s 46ms/step
55
2/2 [==============================] - 0s 4ms/step


c:\Users\lalit\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 0: D Loss Real: 0.7586, D Loss Fake: 0.7456, G Loss: 0.6569
1/1 [==============================] - 0s 49ms/step
56


c:\Users\lalit\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


2/2 [==============================] - 0s 3ms/step
Epoch 0: D Loss Real: 0.6752, D Loss Fake: 0.8193, G Loss: 0.5977
1/1 [==============================] - 0s 47ms/step


In [5]:
synthetic_df

,CROPS,TYPE_OF_CROP,SOIL,SEASON,SOWN,HARVESTED,SOIL_PH,CROP_DURATION,WATERSOURCE,TEMP,WATER_REQUIRED,RELATIVE_HUMIDITY
0,rice,cereals,Alluvia or loamy and clayey soil,kharif,Jun-Jul,Sep-Oct,7.9,150.0,"irrigated,rainfall",37.51,2467.0,79.86
1,rice,cereals,Alluvia or loamy and clayey soil,kharif,Jun-Jul,Sep-Oct,7.9,150.0,"irrigated,rainfall",32.57,2449.0,79.50
2,rice,cereals,Alluvia or loamy and clayey soil,kharif,Jun-Jul,Sep-Oct,7.9,150.0,"irrigated,rainfall",34.78,2445.0,79.56
3,rice,cereals,Alluvia or loamy and clayey soil,kharif,Jun-Jul,Sep-Oct,7.8,150.0,"irrigated,rainfall",30.01,2452.0,79.60
4,rice,cereals,Alluvia or loamy and clayey soil,kharif,Jun-Jul,Sep-Oct,7.9,150.0,"irrigated,rainfall",32.12,2440.0,79.83
...,...,...,...,...,...,...,...,...,...,...,...,...
565,small onion,bulbvegetables,Sandy loam,Zaid,Mar-Jul,Mar-Jul,6.9,82.0,"irrigated,rainfall",16.35,728.0,74.59
566,small onion,bulbvegetables,Sandy loam,Zaid,Mar-Jul,Mar-Jul,6.9,81.0,"irrigated,rainfall",16.44,727.0,74.60
567,small onion,bulbvegetables,Sandy loam,Zaid,Mar-Jul,Mar-Jul,6.9,79.0,"irrigated,rainfall",16.43,728.0,74.37
568,small onion,bulbvegetables,Sandy loam,Zaid,Mar-Jul,Mar-Jul,6.9,82.0,"irrigated,rainfall",16.43,727.0,74.55


In [8]:
# Save synthetic data to CSV
synthetic_df.to_csv("synthetic_crop_data.csv")

In [7]:

synthetic_df.to_csv("crop_data1.csv")